# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [18]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [19]:
sinasc_raw = pd.read_csv('SINASC_RO_2019.csv')
sinasc_raw.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [20]:
# Carregando os dados do arquivo 'SINASC_RO_2019.csv'
df = pd.read_csv('SINASC_RO_2019.csv', delimiter=',')  # Aqui estou assumindo que o delimitador é uma vírgula

# Agrupando pelo município e calculando a média das idades
media_idades = df.groupby('munResNome').agg({
    'IDADEMAE': 'mean',
    'IDADEPAI': 'mean'
}).reset_index()

print(media_idades)

                   munResNome   IDADEMAE   IDADEPAI
0       Alta Floresta D'Oeste  25.991826  29.548701
1     Alto Alegre dos Parecis  24.844156  29.179104
2                Alto Paraíso  24.960177  28.833333
3            Alvorada D'Oeste  25.770732  30.757282
4                   Ariquemes  25.607866  32.466667
5                     Buritis  25.559177  30.812500
6                      Cabixi  26.037500  34.333333
7                 Cacaulândia  25.546667  36.333333
8                      Cacoal  26.890102  30.931854
9      Campo Novo de Rondônia  24.825301  30.500000
10         Candeias do Jamari  25.243323  29.894737
11               Castanheiras  27.281250  30.392857
12                 Cerejeiras  27.240506  31.652778
13                Chupinguaia  25.333333  29.629630
14          Colorado do Oeste  27.602804  34.354839
15                 Corumbiara  24.716535  32.470588
16              Costa Marques  24.412017  30.613636
17                    Cujubim  24.482927  31.400000
18          

### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [21]:
# Verificando o tipo da coluna DTNASC
print(f"Tipo original da coluna DTNASC: {df['DTNASC'].dtype}")

# Convertendo a coluna DTNASC para datetime se for uma string
if df['DTNASC'].dtype == 'object':
    df['DTNASC'] = pd.to_datetime(df['DTNASC'])

# Filtrando os dados pela data de aniversário
aniversario = '2019-01-07'  # Ajuste para a sua data de nascimento, mantendo o ano como exemplo
df_aniversario = df[df['DTNASC'].dt.strftime('%m-%d') == aniversario[5:]]

# Agrupando por sexo e escolaridade da mãe, e calculando a média do peso
peso_medio = df_aniversario.groupby(['SEXO', 'ESCMAE']).agg({
    'PESO': 'mean'
}).reset_index()

print(peso_medio)

Tipo original da coluna DTNASC: object
        SEXO           ESCMAE         PESO
0   Feminino       1 a 3 anos  3340.000000
1   Feminino  12 anos ou mais  2797.857143
2   Feminino       4 a 7 anos  3527.500000
3   Feminino      8 a 11 anos  3180.571429
4  Masculino  12 anos ou mais  3117.777778
5  Masculino       4 a 7 anos  3556.428571
6  Masculino      8 a 11 anos  3444.130435


### 3. Qual o municipio que nasceu menos bebês em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [22]:
# Agrupando os dados pelo município e contando o número de nascimentos
contagem_nascimentos = df.groupby('munResNome').size().reset_index(name='num_nascimentos')

# Identificando o município com o menor número de nascimentos
municipio_menos_nascimentos = contagem_nascimentos.sort_values('num_nascimentos').iloc[0]['munResNome']

print(f"Município com menos nascimentos em 2019: {municipio_menos_nascimentos}")

# Filtrando os registros desse município
df_municipio = df[df['munResNome'] == municipio_menos_nascimentos]

# Calculando as estatísticas para as idades das mães
idade_mae_media = df_municipio['IDADEMAE'].mean()
idade_mae_max = df_municipio['IDADEMAE'].max()
idade_mae_min = df_municipio['IDADEMAE'].min()

print(f"Idade média das mães: {idade_mae_media:.2f}")
print(f"Idade máxima das mães: {idade_mae_max}")
print(f"Idade mínima das mães: {idade_mae_min}")

# Calculando as estatísticas para as idades dos pais
idade_pai_media = df_municipio['IDADEPAI'].mean()
idade_pai_max = df_municipio['IDADEPAI'].max()
idade_pai_min = df_municipio['IDADEPAI'].min()

print(f"Idade média dos pais: {idade_pai_media:.2f}")
print(f"Idade máxima dos pais: {idade_pai_max}")
print(f"Idade mínima dos pais: {idade_pai_min}")

Município com menos nascimentos em 2019: Município ignorado - RO
Idade média das mães: 24.00
Idade máxima das mães: 24
Idade mínima das mães: 24
Idade média dos pais: 22.00
Idade máxima dos pais: 22.0
Idade mínima dos pais: 22.0


### 4. Qual o municipio que nasceu mais bebês no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



In [23]:
# Filtrando os registros de março
df_marco = df[df['DTNASC'].dt.month == 3]

# Agrupando os dados pelo município e contando o número de nascimentos em março
contagem_nascimentos = df_marco.groupby('munResNome').size().reset_index(name='num_nascimentos')

# Identificando o município com o maior número de nascimentos em março
municipio_mais_nascimentos = contagem_nascimentos.sort_values('num_nascimentos', ascending=False).iloc[0]['munResNome']

print(f"Município com mais nascimentos em março: {municipio_mais_nascimentos}")

# Filtrando os registros desse município
df_municipio = df_marco[df_marco['munResNome'] == municipio_mais_nascimentos]

# Calculando as estatísticas para o número de filhos vivos
filhos_vivos_media = df_municipio['QTDFILVIVO'].mean()
filhos_vivos_max = df_municipio['QTDFILVIVO'].max()
filhos_vivos_min = df_municipio['QTDFILVIVO'].min()

print(f"Quantidade média de filhos vivos: {filhos_vivos_media:.2f}")
print(f"Quantidade máxima de filhos vivos: {filhos_vivos_max}")
print(f"Quantidade mínima de filhos vivos: {filhos_vivos_min}")

# Calculando as estatísticas para as idades dos pais
idade_pai_media = df_municipio['IDADEPAI'].mean()
idade_pai_max = df_municipio['IDADEPAI'].max()
idade_pai_min = df_municipio['IDADEPAI'].min()

print(f"Idade média dos pais: {idade_pai_media:.2f}")
print(f"Idade máxima dos pais: {idade_pai_max}")
print(f"Idade mínima dos pais: {idade_pai_min}")

Município com mais nascimentos em março: Porto Velho
Quantidade média de filhos vivos: 1.30
Quantidade máxima de filhos vivos: 9.0
Quantidade mínima de filhos vivos: 0.0
Idade média dos pais: 34.63
Idade máxima dos pais: 62.0
Idade mínima dos pais: 19.0


### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

- O município com menos nascimentos, dá a entender que é um município com baixa população.
- O município com mais nascimentos, descobri que a idade máxima dos pais é de 62 anos.

In [24]:
sinasc_raw.dtypes

ORIGEM          int64
CODESTAB      float64
CODMUNNASC      int64
LOCNASC         int64
IDADEMAE        int64
               ...   
munResUf       object
munResLat     float64
munResLon     float64
munResAlt     float64
munResArea    float64
Length: 69, dtype: object

In [25]:
sinasc_raw.select_dtypes(exclude=['int64','float64'])

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27023,12 anos ou mais,32 a 36 semanas,Única,Cesáreo,2019-12-13,Masculino,Branca,2020-09-28,NaN,3.2.01,2020-10-13,ATIVO,MUNIC,Vilhena,Rondônia
27024,8 a 11 anos,37 a 41 semanas,Única,Vaginal,2019-10-04,Masculino,Branca,2019-10-16,NaN,3.2.01,2019-10-21,ATIVO,MUNIC,Chupinguaia,Rondônia
27025,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-08-02,Masculino,Parda,2019-08-07,NaN,3.2.00,2019-08-07,ATIVO,MUNIC,Vilhena,Rondônia
27026,8 a 11 anos,32 a 36 semanas,Única,Vaginal,2019-12-23,Masculino,Parda,2020-01-07,NaN,3.2.00,2020-01-07,ATIVO,MUNIC,Vilhena,Rondônia


In [26]:
sinasc_raw.columns

Index(['ORIGEM', 'CODESTAB', 'CODMUNNASC', 'LOCNASC', 'IDADEMAE', 'ESTCIVMAE',
       'ESCMAE', 'CODOCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'CODMUNRES',
       'GESTACAO', 'GRAVIDEZ', 'PARTO', 'CONSULTAS', 'DTNASC', 'HORANASC',
       'SEXO', 'APGAR1', 'APGAR5', 'RACACOR', 'PESO', 'IDANOMAL', 'DTCADASTRO',
       'CODANOMAL', 'NUMEROLOTE', 'VERSAOSIST', 'DTRECEBIM', 'DIFDATA',
       'DTRECORIGA', 'NATURALMAE', 'CODMUNNATU', 'CODUFNATU', 'ESCMAE2010',
       'SERIESCMAE', 'DTNASCMAE', 'RACACORMAE', 'QTDGESTANT', 'QTDPARTNOR',
       'QTDPARTCES', 'IDADEPAI', 'DTULTMENST', 'SEMAGESTAC', 'TPMETESTIM',
       'CONSPRENAT', 'MESPRENAT', 'TPAPRESENT', 'STTRABPART', 'STCESPARTO',
       'TPNASCASSI', 'TPFUNCRESP', 'TPDOCRESP', 'DTDECLARAC', 'ESCMAEAGR1',
       'STDNEPIDEM', 'STDNNOVA', 'CODPAISRES', 'TPROBSON', 'PARIDADE',
       'KOTELCHUCK', 'CONTADOR', 'munResStatus', 'munResTipo', 'munResNome',
       'munResUf', 'munResLat', 'munResLon', 'munResAlt', 'munResArea'],
      dtype='object')